### Importamos librerías

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

Lectura

In [2]:
#Leemos el archivo reviews.parquet en un DataFrame
games = pq.read_table("../0-DATA/output_steam_games.parquet").to_pandas()

In [3]:
#Abrimos el DataFrame y analizamos que es lo que tenemos para proceder a la limpieza
games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,None,NaN,None
1,None,None,None,None,None,None,None,None,None,None,None,NaN,None
2,None,None,None,None,None,None,None,None,None,None,None,NaN,None


In [4]:
#vemos las informacion de nuestra tabla, notamos qe existen valores nulos
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  object 
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(1), object(12)
memory usage: 11.9+ MB


### Transformación de nuestra Tabla

Eliminamos duplicados

In [5]:
# analizaremos si tenemos filas duplicadas, 88309 filas duplicadas
duplicados = games.loc[games.duplicated()]
duplicados.shape

(88309, 13)

In [6]:
#Eliminamos los duplicados, solo el 74% del total
games = games.drop_duplicates(keep='first')
games.shape

(32136, 13)

Analizamos ahora si existen nullos

In [7]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32136 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  object 
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(1), object(12)
memory usage: 3.4+ MB


Analizando columnas antes de eliminarlas

In [8]:
games["publisher"].unique() # --> eliminar esta columna es parecida a los desarrolladores y tiene mas nulos
#games["genres"].unique() # no eliminamos por que es importante
#games["app_name"].unique() # es lo mismo que title pero no tiene nulos
#games["title"].unique() # --> eliminar ya que contiene la misma informacion que "app_name"
#games["url"].unique() # --> eliminar por q son links y no nos sirve
#games["release_date"].unique() # --> se elimina por que ya tenemos ese dato en la otra tabla es la fecha de lanzamiento
#games["tags"].unique() # --> eliminamos por q tiene la misma info de gender
#games["reviews_url"].unique() # --> eliminamos por que son puras urls
#games["specs"].unique() # --> lo eliminamos ya que las especificaciones del juego no es de interes en nuestro analisis
#games["price"].unique() # se queda ya que es importante para el tratamiento de datos
#games["early_access"].unique() # --> eliminar,nos nos da una informacion que necesitemos para nuestro modelo, ademas dentro de price tenemos free si el juego es libre
#games["id"].unique() # id unico de jugadores o algo
#games["developer"].unique() #mantenemos es la columna del nombre del desarrollador

array([None, 'Kotoshiro', 'Making Fun, Inc.', ..., 'OrtiGames/OrtiSoft',
       'INGAME', 'Bidoniera Games'], dtype=object)

In [9]:
# Eliminamos las columnas "publisher", "title", "url","tags", "reviews_url", "specs","early_access" tiene informacion no util
games = games.drop(columns=["publisher", "title", "url","tags", "reviews_url", "specs","early_access"])
games.head(3)

,genres,app_name,release_date,price,id,developer
0,None,None,None,None,NaN,None
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL


eliminando datos nulos por cada columna

In [10]:
#Eliminamos las filas de la columna que quedaron vacias - elimina 2 filas
games = games.dropna(subset=['app_name'])
#Eliminamos las filas de la columna que quedaron vacias - elimina 1 fila
games = games.dropna(subset=['id'])
#Eliminamos las filas de la columna que quedaron vacias - elimina 1376 filas
games = games.dropna(subset=['price'])
#Eliminamos las filas de la columna que quedaron vacias - elimina 3135 filas
games = games.dropna(subset=['genres'])
#Eliminamos las filas de la columna que quedaron vacias - elimina 155 filas
games = games.dropna(subset=['developer'])
#Eliminamos las filas de la columna que quedaron vacias - elimina 4 filas
games = games.dropna(subset=['release_date'])

In [11]:
#verificamos si hay mas nulos
nulos = games.isnull().sum() # podemos ver que las columnas 'funny' y 'last_edited', casi en su totalidad son valores nulos
nulos

genres          0
app_name        0
release_date    0
price           0
id              0
developer       0
dtype: int64

In [12]:
# vemos que nos quedamos con 27466 filas con informacion valiosa (23% de la data total) puros nulos en la tabla 77%
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27462 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27462 non-null  object 
 1   app_name      27462 non-null  object 
 2   release_date  27462 non-null  object 
 3   price         27462 non-null  object 
 4   id            27462 non-null  float64
 5   developer     27462 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.5+ MB


### Modificando la columnas 

'release_date'

In [13]:
#composición de la columna release_date
games["release_date"]

88310     2018-01-04
88311     2018-01-04
88312     2017-07-24
88313     2017-12-07
88315     2018-01-04
             ...    
120439    2018-01-04
120440    2018-01-04
120441    2018-01-04
120442    2018-01-04
120443    2017-09-02
Name: release_date, Length: 27462, dtype: object

In [14]:

# Extraemos el año de la columna "release_date" 
games['Año_estreno'] = games['release_date'].str.extract(r'(\d{4})')
games.head(5)

,genres,app_name,release_date,price,id,developer,Año_estreno
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro,2018
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL,2018
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com,2017
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域,2017
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,3.99,772540.0,Trickjump Games Ltd,2018


In [15]:
#eliminamos la columna 'release_date'
games = games.drop('release_date', axis=1)
games.head(5)

,genres,app_name,price,id,developer,Año_estreno
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Free To Play,643980.0,Secret Level SRL,2018
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Free to Play,670290.0,Poolians.com,2017
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400.0,彼岸领域,2017
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018


'price'

In [16]:
#composición de la columna "price"
games["price"].unique()

array(['4.99', 'Free To Play', 'Free to Play', '0.99', '3.99', '9.99',
       '18.99', '29.99', '10.99', '2.99', '1.59', '14.99', '1.99',
       '59.99', '8.99', '6.99', '7.99', '39.99', 'Free', '19.99', '7.49',
       '12.99', '5.99', '2.49', '15.99', '1.25', '24.99', '17.99',
       '61.99', '3.49', '11.99', '13.99', 'Free Demo', 'Play for Free!',
       '34.99', '1.49', '32.99', '99.99', '14.95', '69.99', '16.99',
       '79.99', '49.99', '5.0', '13.98', '29.96', '109.99', '149.99',
       '771.71', 'Install Now', '21.99', '89.99',
       'Play WARMACHINE: Tactics Demo', '0.98', '139.92', '4.29',
       'Free Mod', '54.99', '64.99', '74.99', 'Install Theme', '0.89',
       'Third-party', '0.5', 'Play Now', '299.99', '1.29', '119.99',
       '44.99', '3.0', '15.0', '5.49', '23.99', '49.0', '10.93', '1.39',
       'Free HITMAN™ Holiday Pack', '36.99', '4.49', '2.0', '4.0', '1.95',
       '1.5', '199.0', '189.0', '6.66', '27.99', '129.99', '179.0',
       '26.99', '399.99', '31.99', '3

In [17]:
# definimos una función para poder reemplazar los datos string de la columna "price" por flotantes

def cambiando_tipo_variable(valor):
    try:
        flotante = float(valor) #si el valor es float lo conservo
        return flotante
    except (ValueError, TypeError): #si el valor es un string entonces retorno 0.0
        return 0.0
    
#Aplico esta función a la columna precio
games["price"] = games["price"].apply(cambiando_tipo_variable)

In [18]:
#vemos de nuevo la nueva composición de la columna "price"
games["price"].unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 3.9900e+00, 9.9900e+00,
       1.8990e+01, 2.9990e+01, 1.0990e+01, 2.9900e+00, 1.5900e+00,
       1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00, 6.9900e+00,
       7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00, 1.2990e+01,
       5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00, 2.4990e+01,
       1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01, 1.3990e+01,
       3.4990e+01, 1.4900e+00, 3.2990e+01, 9.9990e+01, 1.4950e+01,
       6.9990e+01, 1.6990e+01, 7.9990e+01, 4.9990e+01, 5.0000e+00,
       1.3980e+01, 2.9960e+01, 1.0999e+02, 1.4999e+02, 7.7171e+02,
       2.1990e+01, 8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00,
       5.4990e+01, 6.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01,
       2.9999e+02, 1.2900e+00, 1.1999e+02, 4.4990e+01, 3.0000e+00,
       1.5000e+01, 5.4900e+00, 2.3990e+01, 4.9000e+01, 1.0930e+01,
       1.3900e+00, 3.6990e+01, 4.4900e+00, 2.0000e+00, 4.0000e+00,
       1.9500e+00, 1.5000e+00, 1.9900e+02, 1.8900e+02, 6.6600e

In [19]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27462 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   genres       27462 non-null  object 
 1   app_name     27462 non-null  object 
 2   price        27462 non-null  float64
 3   id           27462 non-null  float64
 4   developer    27462 non-null  object 
 5   Año_estreno  27459 non-null  object 
dtypes: float64(2), object(4)
memory usage: 1.5+ MB


Reseteamos el index, ya que hemos eliminado filas

In [20]:
#Reseteamos los indices
games.reset_index(inplace=True)
games.head(3)

,index,genres,app_name,price,id,developer,Año_estreno
0,88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980.0,Secret Level SRL,2018
2,88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017


In [21]:
#eliminamos el index antiguo que se convirtion en una columna
games = games.drop('index', axis=1)
games.head(5)

,genres,app_name,price,id,developer,Año_estreno
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980.0,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400.0,彼岸领域,2017
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018


Guardamos el ETL el archivos CSV y Parquet

In [22]:
tabla = pa.Table.from_pandas(games) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/output_steam_gamesETL.parquet") # guardamos la tabla en formato parquet

In [23]:
games.to_csv("../0-DATA/output_steam_gamesETL.csv", index=False, encoding="utf-8") 